Import the required libraries.

In [ ]:
from pynucseg import ReferenceImage # or import pynucseg
import numpy as np
import pandas as pd  # "!pip install pandas"
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import plotly.express as px # !pip install plotly nbformat
import plotly.io as pio # kaleido is required. TO install >>> !pip install -U kaleido
import seaborn as sns
import shelve
import tifffile

Define file path and names. 

In [ ]:
root_path = 'E:/Lab/' # 'D:/User Folders/Ganesh/' 
file_path = 'Data/H3K27ac + PolII/Effect of fixation method/Intensity measurement in PBS/230503 v6.5 P2 D1/'

# output_path = "Python output".join(file_path.split('SRX output'))
output_path = file_path
n_fov = 6  # number of filed of views
start_fov = 4  # by default it should be 0
n_channel = 2  # number of channels with fluorescent nucleus 
field_of_view='f' # the very first prefix of the file name
probe1_name = 'H3K27ac-AF647' # 'H2B', 'H3K36me3
probe2_name = 'PolII-AF488'
transmitted_name ='t'
imaging_setup = {'Laser': ['640nm', ['488nm']],
                 'Power': ["0.1'%' of 60%","0.1'%' of 60%" ],
                 'Exposure': ['100ms', '100ms'],
                 }

Construct ReferenceImage objects and store images, perform smoothing and intensity computations inplace.

In [ ]:
for fov in range(start_fov,start_fov+n_fov):
    img = ReferenceImage(root_path=root_path,
                   file_path=file_path,
                   field_of_view=field_of_view+str(fov+1),
                   probe1_name=probe1_name,
                   probe2_name=probe2_name,
                   transmitted_name='t',
                   imaging_setup=imaging_setup,
                   )
    # median filtering to smooth out
    img.smoothing(image_name=probe2_name,sigma=8)

    # run segmentation
    img.run_segmentaion2d(prob_thresh=0.5,
                          nms_threhold=0.01,
                          scale=0.2,  # scale down the image prior segmentation,
                          )

    # get measuremtns, save result inplace, do not spit out
    # default area per pixel is 0.01 um^2 assuming 1 pixel=100nm
    # Area_threshold = [amin, amax]. Nuclei with area smaller than amin and 
    # larger than amax will be rejected
    img.get_cell_info(probe_names=[probe1_name,probe2_name],
                      a_pixel_area=0.01,area_threshold=[65,200]) 

List out all the fovs of ReferenceImage

In [ ]:
imgs_list = ReferenceImage.all_fovs

Contour centers.

In [ ]:
all_contour_center = {}
for fov in range(start_fov,start_fov+n_fov):
        # current field of view (or image)
        img = imgs_list[fov-start_fov]

        contour_centers = np.zeros((img.n_true_nucleus,2)) 
        for i,ind in enumerate(img.true_nucleus):
                # print(i,ind)
                contour_centers[i,:] = np.mean(img.contours[int(ind[0])],axis=0)
        
        all_contour_center[fov] = contour_centers

Plot the images and the (background and area filtered ) contours. 

In [ ]:
%%capture
# %%capture supress the printouts of figure on jupyter notebook

# Adjust brightness and contrast / set normalizations ADU count
vmin0, vmax0 = 2000, 30000  # For Transmitted light image
vmin1, vmax1 = 150, 3000  # For Probe 1 images
vmin2, vmax2 = 150, 1500  # For probe 2 images


fontsize = 20
for fov in range(start_fov,start_fov+n_fov):

        # current field of view (or image)
        img = imgs_list[fov-start_fov]

        fig, axs = plt.subplots(1,3, figsize=(25,10));
        # Hide the axis and label
        for ax in axs:
                ax.axis('off')
                # Plot contours in all subplots
                for i,ind in enumerate(img.true_nucleus):
                        ax.plot(img.contours[int(ind[0])][:,1],
                                img.contours[int(ind[0])][:,0]);

        # Plot transmitted Image 
        axs[0].imshow(img.image['t'],
                      cmap='gray',vmin=vmin0,vmax=vmax0);
        axs[0].set_title('Transmitted (w/ cell_ID and area in $\mu m^2$)',fontsize=fontsize);

        for i,ind in enumerate(img.true_nucleus):
                text_to_show = f"ID:{int(ind[0])}\n{round(img.seg['area_n_bdry_filtered_area_in_um2'][i,0],2)}"
                axs[0].text(all_contour_center[fov][i,1], 
                            all_contour_center[fov][i,0], 
                            text_to_show, color = 'k', fontweight='bold',
                            bbox=dict(facecolor='w', edgecolor='none', alpha=0.3),
                            fontsize=8, ha='center', va='center');

        # Plot Probe 1
        axs[1].imshow(img.image[probe1_name],
                      cmap='gray', vmin=vmin1,vmax=vmax1); # norm=colors.LogNorm(vmin=100,vmax=5000)
        axs[1].set_title(f"{probe1_name} w/ meanADU/px [Norm:{vmin1}-{vmax1}]",fontsize=fontsize);
        for i,ind in enumerate(img.true_nucleus):
                text_to_show = f"ID:{int(ind[0])}\n{round(img.ADU_counts[i,0])}"
                axs[1].text(all_contour_center[fov][i,1],
                            all_contour_center[fov][i,0], 
                            text_to_show,color = 'k', fontweight='bold',
                            bbox=dict(facecolor='w', edgecolor='none', alpha=0.3),
                            fontsize=8, ha='center', va='center');

        # Plot probe 2
        axs[2].imshow(img.image[probe2_name],
                      cmap='gray', vmin=vmin2,vmax=vmax2);
        axs[2].set_title(f"{probe2_name} w/ meanADU/px [Norm:{vmin2}-{vmax2}]",fontsize=fontsize);
        for i,ind in enumerate(img.true_nucleus):
                text_to_show = f"ID:{int(ind[0])}\n{round(img.ADU_counts[i,1])}"
                axs[2].text(all_contour_center[fov][i,1],
                            all_contour_center[fov][i,0],
                            text_to_show,color = 'k', fontweight='bold',
                            bbox=dict(facecolor='w', edgecolor='none', alpha=0.3),
                            fontsize=8, ha='center', va='center');
        fig.tight_layout();
        fig.suptitle('FOV: '+str(fov+1), fontsize=fontsize+2, fontweight='bold')
        fig.savefig(root_path+output_path+'Segmented_img f'+str(fov+1)+'.png',dpi=300);

Extract result to DataFrame, and save to disk as csv.

In [ ]:
# # Create an empty Pandas data frame with column names
# df = pd.DataFrame(columns=['FOV_ID', 'Cell_ID','Area_px' ,'Area_um2',probe1_name,probe2_name])

# for fov in range(start_fov,start_fov+n_fov):

#     # current field of view (or image)
#     img = imgs_list[fov-start_fov]

#     # add FOV_ID and Cell_ID column in ADU_count
#     data = np.hstack(
#         (np.repeat(fov+1,img.n_true_nucleus).reshape(-1,1),
#          img.seg['area_n_bdry_filtered_IDs'],
#          img.seg['area_n_bdry_filtered_area'],
#          img.seg['area_n_bdry_filtered_area_in_um2'],
#          img.ADU_counts,
#          ))
#     # # concatenate  the values in the main dataframe
#     df = pd.concat([df, pd.DataFrame(data=data, columns=df.columns)], ignore_index=True)

# # remove #.0 type data to pure int type
# df['FOV_ID'] = df['FOV_ID'].astype(int)  
# df['Cell_ID'] = df['Cell_ID'].astype(int)  

# # convert int to str type to use number as categorical
# df['FOV_ID'] = df['FOV_ID'].astype(str)  #  str make FOV_ID categorical
# df['Cell_ID'] = df['Cell_ID'].astype(str)  #  str make FOV_ID categorical

# # save DataFrame to disk as csv
# df.to_csv(root_path+output_path+'mean_intensity_per_pxl.csv',index=False)

Compute mean and standard error of measuremt.

In [ ]:
# stat = df[[probe1_name,probe2_name]].describe() # get count, mean, std, min, 25%, 50%, 75%, max
# n_cells = stat.loc['count'][0] # number of cells, i.e. data points
# std_err = stat.loc['std']/np.sqrt(n_cells) # standard error

Violin, Box, and Scatter plot for mean intensities per nucleus per probe. 

In [ ]:
# title = 'Intensity comparision on two color channels'
# fig, ax = plt.subplots()
# # ax = sns.violinplot(data=df[[probe1_name,probe2_name]], inner=None, linewidth=1, saturation=0.5,alpha=0)

# # # change alpha for edges and faces
# # ax.collections[0].set_edgecolor((1,0,1, 1))
# # ax.collections[1].set_edgecolor((0,1, 1,1))
# # ax.collections[0].set_facecolor((1,0,1, 0.05))
# # ax.collections[1].set_facecolor((0,1,1, 0.05))

# sns.boxplot(data=df[[probe1_name,probe2_name]], saturation=0.5, width=0.2,
#             palette='rocket', boxprops={'zorder': 2,'facecolor': 'None'},  ax=ax,
#             showfliers=False)

# sns.stripplot(data=df[[probe1_name,probe2_name]], jitter=True)


# ax.plot([0,1],stat.loc['mean'],'sg')
# # Add text to the plot
# plt.text(0+0.1, stat.loc['mean'][probe1_name], 
#         f"$\mu=${round(stat.loc['mean'][probe1_name])}"+\
#         u"\u00B1"+\
#         f"{round(std_err[probe1_name])}")  # u"\u00B1" = +- symbol
# plt.text(1+0.1, stat.loc['mean'][probe2_name], 
#         f"$\mu=${round(stat.loc['mean'][probe2_name])}"+\
#         u"\u00B1"+\
#         f"{round(std_err[probe2_name])}")

# # adjust axis limits
# # ax.set_ylim(-100,3000)
# ax.set_xlim(-0.5,1.5)
# ax.set_ylabel('Mean intensity per pixel')
# ax.set_title(title+f'\n(n={int(n_cells)})')
# # show plot
# plt.savefig(root_path+output_path+ title+'.png')
# # plt.show();

Area versus intensity per probe.

In [ ]:
# fig = px.scatter(data_frame=df,
#                  x='Area_um2',
#                  y=[probe1_name],
#                  color='FOV_ID',
#                  hover_data=['FOV_ID','Cell_ID'])
# fig.update_layout(
#     yaxis_title='Mean intensity per pixel',
#     xaxis_title='Area of cell in um^2',
#     title=dict(
#         text=probe1_name+' intensity versus nucleus size',
#         x=0.5,
#         y=0.95,
#         xanchor="center",
#         yanchor="top",
#         ),
# )
# pio.write_image(fig, root_path+output_path+'area_vs_intensity '+probe1_name+'.png',format='png')
# # fig.show()

In [ ]:
# fig = px.scatter(data_frame=df,
#                  x='Area_um2',
#                  y=[probe2_name],
#                  color='FOV_ID',
#                  hover_data=['FOV_ID','Cell_ID'])
# fig.update_layout(
#     yaxis_title='Mean intensity per pixel',
#     xaxis_title='Area of cell in um^2',
#     title=dict(
#         text=probe2_name+' intensity versus nucleus size',
#         x=0.5,
#         y=0.95,
#         xanchor="center",
#         yanchor="top",
#         ),
# )
# pio.write_image(fig, root_path+output_path+'area_vs_intensity '+probe2_name+'.png',format='png')
# # fig.show()

Store the masks images in the disk.

In [ ]:
for fov in range(start_fov,start_fov+n_fov):
    # current field of view (or image)
    img = imgs_list[fov-start_fov]

    # tifffile.imwrite(root_path+output_path+'labels_stardist f'+ str(fov+1)+'.tif',
    #                  img.image['labels'],dtype=np.uint8)
    
    # tifffile.imwrite(root_path+output_path+'labels_boundary_filtered f'+ str(fov+1)+'.tif',
    #                  img.image['boundary_filtered_labels'],dtype=np.uint8)
    
    # save the segmented cells labels as 8-bit image
    tifffile.imwrite(root_path+output_path+'labels_area_filtered f'+ str(fov+1)+'.tif',
                     img.image['area_filtered_labels'],dtype=np.uint8)
    
    # save the background mask as binary image
    tifffile.imwrite(root_path+output_path+'labels_background f'+ str(fov+1)+'.tif',
                     img.image['background_label'],dtype=bool)

View images and masks in napari.

In [ ]:
# fov_to_visualize = 2
# # to print list of names of available images
# imgs_list[fov_to_visualize-1].show_image_names()


# # display images in napari
# viewer1 = imgs_list[fov_to_visualize-1].view_on_napari(return_viewer=True)

# # the last field of view
# # img.view_on_napari()


Save the `'imgs_list'` as pickle file.

In [ ]:
# results = shelve.open(root_path+output_path+'results', writeback=True)
# results['imgs_list'] = imgs_list
# results.sync()
# results.close()

Open the saved `results` and visualize the images, masks, and the background.

In [ ]:
# with shelve.open('E:/Lab/Data/H2B + PolII/Python output/Intensity measurement in PBS/230318 V6.5 P14 D3/results') as results_new:
#     imgs_list_new = results_new['imgs_list']


In [ ]:
# fov_to_visualize = 1

# # display images in napari
# imgs_list_new[fov_to_visualize-1].view_on_napari()